In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from typing import List, Literal, TypedDict, Annotated
from langchain_openai import ChatOpenAI
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import BaseTool, StructuredTool, tool
from langchain.prompts import PromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolInvocation, ToolExecutor

# Define our State
class State(TypedDict):
    task: str
    history: List[BaseMessage]
    agent_outcome: Annotated[str | ToolInvocation, "The most recent outcome from the agent"]

In [3]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Union
from langchain.tools import BaseTool

class PrimeCalculatorInput(BaseModel):
    limit: Union[int, str] = Field(..., description="The upper limit for calculating prime numbers")

    @field_validator('limit')
    @classmethod
    def limit_must_be_positive(cls, v):
        if isinstance(v, str):
            try:
                v = int(v)
            except ValueError:
                raise ValueError("Limit must be a valid integer")
        if v <= 0:
            raise ValueError("Limit must be a positive integer")
        return v

class PrimeCalculator(BaseTool):
    name = "prime_calculator"
    description = "Calculate prime numbers up to a given limit and their sum"
    args_schema = PrimeCalculatorInput

    def _run(self, limit: Union[int, str]) -> str:
        limit = int(limit)  # Convert to int if it's a string
        def is_prime(n):
            if n < 2:
                return False
            for i in range(2, int(n ** 0.5) + 1):
                if n % i == 0:
                    return False
            return True

        primes = [n for n in range(2, limit) if is_prime(n)]
        return f"Prime numbers up to {limit}: {primes}\nSum of these primes: {sum(primes)}"

class StatisticsCalculatorInput(BaseModel):
    numbers: str = Field(..., description="Comma-separated list of numbers")

    @field_validator('numbers')
    @classmethod
    def validate_numbers(cls, v):
        try:
            nums = [float(n.strip()) for n in v.split(',')]
            if not nums:
                raise ValueError("Please provide a non-empty list of numbers")
        except ValueError:
            raise ValueError("Invalid input. Please provide a comma-separated list of numbers")
        return v

class StatisticsCalculator(BaseTool):
    name = "statistics_calculator"
    description = "Calculate basic statistics (mean, median, mode) for a given list of numbers"
    args_schema = StatisticsCalculatorInput

    def _run(self, numbers: str) -> str:
        nums = [float(n.strip()) for n in numbers.split(',')]
        mean = sum(nums) / len(nums)
        sorted_nums = sorted(nums)
        median = sorted_nums[len(nums) // 2] if len(nums) % 2 != 0 else (sorted_nums[len(nums) // 2 - 1] + sorted_nums[len(nums) // 2]) / 2
        mode = max(set(nums), key=nums.count)
        return f"Mean: {mean}\nMedian: {median}\nMode: {mode}"

class TextAnalyzerInput(BaseModel):
    text: str = Field(..., description="Text to analyze")

    @field_validator('text')
    @classmethod
    def text_must_not_be_empty(cls, v):
        if not v.strip():
            raise ValueError("Please provide a non-empty text to analyze")
        return v

class TextAnalyzer(BaseTool):
    name = "text_analyzer"
    description = "Analyze text to count words, characters, and most common words"
    args_schema = TextAnalyzerInput

    def _run(self, text: str) -> str:
        words = text.split()
        char_count = len(text)
        word_count = len(words)
        word_freq = {}
        for word in words:
            word_freq[word] = word_freq.get(word, 0) + 1
        common_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:5]
        return f"Character count: {char_count}\nWord count: {word_count}\nMost common words: {common_words}"

# Create instances of our tools
tools = [
    PrimeCalculator(),
    StatisticsCalculator(),
    TextAnalyzer()
]

In [4]:

# Create the language model
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# Create the ReAct agent prompt
prompt = PromptTemplate.from_template("""You are Meta-Expert, an AI assistant designed to tackle complex tasks by breaking them down, using available tools, and coordinating with expert models when needed.

Your task: {task}

To help you, you have access to the following tools:

{tools}

Available tool names: {tool_names}

To use a tool, format your response as follows:
Action: <tool_name>
Action Input: <input_as_json>

For example:
Action: prime_calculator
Action Input: {{"limit": "100"}}

Note: Provide the input as a JSON object with the appropriate field names. For numerical inputs, you can provide them as strings (e.g., "100" instead of 100).

You can also consult with expert models by specifying the type of expert you need (e.g., "Mathematician", "Physicist", "Philosopher", etc.).

Use the following format for your overall response:

Thought: Think about what to do
Action: Choose an action (either a tool name or "Consult Expert")
Action Input: The input to the action
Observation: The result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Action: Answer
Action Input: The final answer

Begin!

{history}
Human: {input}
AI: Certainly! Let me approach this task step by step.

Thought: To tackle this problem, I should first understand what's being asked and then use the appropriate tools or consult experts as needed.

{agent_scratchpad}""")

# Create the ReAct agent
agent = create_react_agent(llm, tools, prompt)

# Create an agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [5]:
def run_agent(state: State) -> State:
    task = state["task"]
    history = state["history"]
    
    # Prepare the input for the agent
    agent_input = {
        "task": task,
        "history": "\n".join([f"{m.type}: {m.content}" for m in history]),
        "input": "Please continue with the task."
    }
    
    try:
        # Run the agent
        result = agent_executor.invoke(agent_input)
        
        # Update the state with the agent's output
        return {
            "task": state["task"],
            "history": state["history"] + [HumanMessage(content=str(agent_input)), AIMessage(content=result["output"])],
            "agent_outcome": result["output"]
        }
    except Exception as e:
        error_message = f"An error occurred while running the agent: {str(e)}"
        return {
            "task": state["task"],
            "history": state["history"] + [HumanMessage(content=str(agent_input)), AIMessage(content=error_message)],
            "agent_outcome": error_message
        }

In [6]:
def should_end(state: State) -> Literal["run_agent", "end"]:
    if "Final Answer:" in state["agent_outcome"]:
        return "end"
    if "An error occurred" in state["agent_outcome"]:
        print(f"Error encountered: {state['agent_outcome']}")
        return "end"
    return "run_agent"

In [7]:
# Create the graph
workflow = StateGraph(State)

# Add the agent node to the graph
workflow.add_node("run_agent", run_agent)

# Set the entry point
workflow.set_entry_point("run_agent")

# Add a conditional edge
workflow.add_conditional_edges(
    "run_agent",
    should_end,
    {
        "run_agent": "run_agent",
        "end": END
    }
)

# Compile the graph
app = workflow.compile()

In [8]:
def main():
    task = "Analyze the prime numbers below 100, calculate their sum, and discuss the significance of this sum in number theory. Also, provide some statistical insights about these prime numbers."
    initial_state = State(task=task, history=[], agent_outcome="")

    # Run the graph
    for output in app.stream(initial_state):
        # The output is a dictionary with a single key (the node name) and its corresponding output
        node_name = list(output.keys())[0]
        node_output = output[node_name]
        
        if node_name == END:
            print("\nFinal Answer:", node_output["agent_outcome"])
            break
        else:
            print(f"\nStep {node_name}:")
            print(node_output["agent_outcome"])

    print("\nMeta-prompting process completed.")


In [9]:
main()



> Entering new AgentExecutor chain...
Thought: The first step is to calculate the prime numbers below 100 and their sum. For this, I will use the prime_calculator tool.

Action: prime_calculator
Action Input: {"limit": "100"}Error encountered: An error occurred while running the agent: invalid literal for int() with base 10: '{"limit": "100"}'

Step run_agent:
An error occurred while running the agent: invalid literal for int() with base 10: '{"limit": "100"}'

Meta-prompting process completed.
